In [3]:
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

from ultralytics import YOLO
import numpy as np
import cv2
import time
import pandas as pd
import csv
import os
import pickle
import warnings

# --- Configuration ---
FIXED_SIZE = 640
SMALL_SIZE = 150, 150
NUM_FRAMES = 15
model_yolo = YOLO("../../model/yolo/yolo12n.pt") 

with open('../../model/trained/svc/svc_model_v1.pkl', 'rb') as f:
    model_pred = pickle.load(f)

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [4]:
def cropped_frame(frame):
    results = model_yolo(frame, classes=[0], verbose=False)
    boxes = results[0].boxes
    plotted_frame = results[0].plot() 

    try:
        if len(boxes) > 0:
            # Get the bounding box coordinates for the first detected object
            x1, y1, x2, y2 = boxes.xyxy[0].cpu().numpy().astype(int)

            cropped_frame = frame[y1:y2, x1:x2]
            crop_h, crop_w = cropped_frame.shape[:2]

            # We want to fit the largest dimension (width or height) to the FIXED_SIZE
            scale = FIXED_SIZE / max(crop_w, crop_h)
            new_w = int(crop_w * scale)
            new_h = int(crop_h * scale)

            # Resize the cropped frame to the new dimensions
            resized_img = cv2.resize(cropped_frame, (new_w, new_h), interpolation=cv2.INTER_AREA)

            # Background frame, that are not filled with boxes
            final_frame = np.full((FIXED_SIZE, FIXED_SIZE, 3), 255, dtype=np.uint8)
            
            # dw and dh are the space left over after placing the image
            dw = FIXED_SIZE - new_w
            dh = FIXED_SIZE - new_h

            # Calculate the starting position (top-left corner) for centering
            top = dh // 2
            bottom = top + new_h
            left = dw // 2
            right = left + new_w

            final_frame[top:bottom, left:right] = resized_img

            return final_frame

        else:
            print("No objects detected in the image.")
    except Exception as e:
        print(f"Error processing frame: {e}")

In [5]:
def side_state(state, lndmrkX):
    if state == 0:
        if lndmrkX["nose"] == min(lndmrkX.values()):
            direction = 1 # Kiri
        elif lndmrkX["nose"] == max(lndmrkX.values()):      
            direction = 2 # Kanan
        else:
            direction = 0 # Tengah
    else:
        if lndmrkX["nose"] == min(lndmrkX.values()):
            direction = 2 # Kanan
        elif lndmrkX["nose"] == max(lndmrkX.values()):      
            direction = 1 # Kiri
        else:
            direction = 0 # Tengah
    
    return direction

def hand_state(state, lndmrkZ):
    if state == 0:
        if lndmrkZ["wrist_r"] and lndmrkZ["wrist_l"] < lndmrkZ["nose"]:
            hand = 1 # Terlihat
        else:
            hand = 0 # Tidak Terlihat
    else:
        if lndmrkZ["wrist_r"] and lndmrkZ["wrist_l"] < lndmrkZ["nose"]:
            hand = 0 # Terlihat
        else:
            hand = 1 # Tidak Terlihat
    return hand

def get_extFeature_value(lndmrk):
    noseX, noseY, noseZ = lndmrk["nose"].x, lndmrk["nose"].y, lndmrk["nose"].z
    earLX, earLY, earLZ = lndmrk["ear_l"].x, lndmrk["ear_l"].y, lndmrk["ear_l"].z
    earRX, earRY, earRZ = lndmrk["ear_r"].x, lndmrk["ear_r"].y, lndmrk["ear_r"].z
    wristRX, wristRY, wristRZ = lndmrk["wrist_r"].x, lndmrk["wrist_r"].y, lndmrk["wrist_r"].z
    wristLX, wristLY, wristLZ = lndmrk["wrist_l"].x, lndmrk["wrist_l"].y, lndmrk["wrist_l"].z
    
    lndmrkX = {"nose": noseX, "ear_l": earLX, "ear_r": earRX, "wrist_r": wristRX, "wrist_l": wristLX }
    lndmrkY = {"nose": noseY, "ear_l": earLY, "ear_r": earRY, "wrist_r": wristRY, "wrist_l": wristLY }
    lndmrkZ = {"nose": noseZ, "ear_l": earLZ, "ear_r": earRZ, "wrist_r": wristRZ, "wrist_l": wristLZ }

    if noseZ < (earLZ and earRZ):
        state = 0
        side = side_state(0, lndmrkX)
        hand = hand_state(0, lndmrkZ)
    else:
        state = 1
        side = side_state(1, lndmrkX)
        hand = hand_state(1, lndmrkZ)

    return side, state, hand



In [16]:
# cap = cv2.VideoCapture('http://192.168.100.197:5000/video')
cap = cv2.VideoCapture(f'../../assets/dataset/v1/nodong_1.mp4')

frames = []
detected = []

# ==== EXTENDED FEATURE ====
face_direction = []
face_shown = []
hand_shown = []

undetectable_image = None
final_layout = None 

saved = 0

if not cap.isOpened():
    print("Error opening video file")
else:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            start_time = time.time()
            ret, frame = cap.read()
            if ret:
                frame_cropped = cropped_frame(frame)
                
                # FIX 1: Skip if initial cropping failed (frame_cropped is None)
                if frame_cropped is None:
                    print("Skipping frame: cropped_frame returned None.")
                    continue

                # Prepare the current frame for side display (resized) and an empty list for detection
                current_small_frame = cv2.resize(frame_cropped, (SMALL_SIZE))
                current_detection_list = []
                detection_successful = False

                try:
                    # Attempt Mediapipe detection
                    frames_mp = cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2RGB)
                    frames_mp.flags.writeable = False
                    
                    results = holistic.process(frames_mp)
                    frames_mp.flags.writeable = True
                    frames_mp = cv2.cvtColor(frame_cropped, cv2.COLOR_RGB2BGR) 

                    # Extract landmarks if available
                    if results.pose_landmarks:
                        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
                        ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
                        ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
                        wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
                        wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

                        current_detection_list.append(nose)
                        current_detection_list.append(ear_l)
                        current_detection_list.append(ear_r)
                        current_detection_list.append(wrist_r)
                        current_detection_list.append(wrist_l)
                        detection_successful = True
                    else:
                        raise ValueError("No pose landmarks detected.")
                        
                except Exception as e:
                    # Detection failed (Mediapipe error or no landmarks found)
                    error_text = f"No detection: {e}"
                    cv2.putText(frame_cropped, error_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    
                if detection_successful:
                    # This block is now ONLY executed if detection_successful is True
                    extended_feature = get_extFeature_value({"nose": nose, "ear_r": ear_r, "ear_l": ear_l, "wrist_r": wrist_r, "wrist_l": wrist_l})

                    if len(frames) < NUM_FRAMES:
                        frames.append(current_small_frame)

                        face_direction.append(extended_feature[0])
                        face_shown.append(extended_feature[1])
                        hand_shown.append(extended_feature[2])

                        detected.append(current_detection_list)
                    else:
                        frames.pop(0)

                        face_direction.pop(0)
                        face_shown.pop(0)
                        hand_shown.pop(0)

                        detected.pop(0)
                        
                        frames.append(current_small_frame)

                        face_direction.append(extended_feature[0])
                        face_shown.append(extended_feature[1])
                        hand_shown.append(extended_feature[2])

                        detected.append(current_detection_list)
                else:
                    print(f"Skipping frame {time.time() - start_time}ms: No pose landmarks detected.")
                    pass
                
                # --- LAYOUT AND DISPLAY LOGIC ---
                if len(frames) == NUM_FRAMES:
                    stacked1_frames = np.vstack((frames[0], frames[1], frames[2]))
                    stacked2_frames = np.vstack((frames[3], frames[4], frames[5]))
                    stacked3_frames = np.vstack((frames[6], frames[7], frames[8]))
                    stacked4_frames = np.vstack((frames[9], frames[10], frames[11]))
                    stacked5_frames = np.vstack((frames[12], frames[13], frames[14]))

                    stacked1_frames = cv2.resize(stacked1_frames, (SMALL_SIZE[0], FIXED_SIZE))
                    stacked2_frames = cv2.resize(stacked2_frames, (SMALL_SIZE[0], FIXED_SIZE))
                    stacked3_frames = cv2.resize(stacked3_frames, (SMALL_SIZE[0], FIXED_SIZE))
                    stacked4_frames = cv2.resize(stacked4_frames, (SMALL_SIZE[0], FIXED_SIZE))
                    stacked5_frames = cv2.resize(stacked5_frames, (SMALL_SIZE[0], FIXED_SIZE))

                    # All components are guaranteed to be valid images here
                    final_layout = np.hstack((frame_cropped, stacked1_frames, stacked2_frames, stacked3_frames, stacked4_frames, stacked5_frames))
                else:
                    final_layout = frame_cropped
                
                if final_layout is not None and final_layout.size > 0:
                    flat_detected = [landmark for sublist in detected for landmark in sublist]
                    # print(len(flat_detected)) # Should always be 5 * NUM_FRAMES when full

                    if len(flat_detected) > 0:
                        landmark_list = landmark_pb2.NormalizedLandmarkList()
                        landmark_list.landmark.extend(flat_detected)
                    
                    try:
                        # motion_row = []
                        
                        # for lv in landmark_list.landmark:
                        #     motion_row.extend([lv.x, lv.y, lv.z, lv.visibility])

                        counter = 0
                        featurePer_frames = 0
                        motion_row = []

                        for lndmrk in landmark_list.landmark:
                            motion_row.append(lndmrk.x)
                            motion_row.append(lndmrk.y)
                            motion_row.append(lndmrk.z)
                            motion_row.append(lndmrk.visibility)
                            counter += 1

                            if counter % 5 == 0:
                                motion_row.append(face_direction[featurePer_frames])
                                motion_row.append(face_shown[featurePer_frames])
                                motion_row.append(hand_shown[featurePer_frames])
                                featurePer_frames += 1

                        motion_row = list(np.array(motion_row))

                        motion_detected = pd.DataFrame([motion_row])
                        motion_class = model_pred.predict(motion_detected)[0]

                        warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
                        cv2.putText(final_layout, f'Class: {motion_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                        cv2.imshow('Video', final_layout)

                        key = cv2.waitKey(1) & 0xFF

                        if key == ord('q'):
                            break
                    
                            
                    except Exception as e:
                        print(f"Error displaying frame: {e}")
                
            else:
                break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1761545885.763451   63862 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1761545885.772291   69817 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: llvmpipe (LLVM 20.1.2, 256 bits)
W0000 00:00:1761545886.062978   69805 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761545886.241789   69809 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761545886.256370   69816 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761545886.258436   69815 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W

Error displaying frame: X has 23 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 46 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 69 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 92 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 115 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 138 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 161 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 184 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 207 features, but StandardScaler is expecting 345 features as input.
Error displaying frame: X has 230 features, but StandardScaler is expecting 345 features as input.
Error displayi